<a href="https://colab.research.google.com/github/zhangxs131/Transformers_example/blob/main/%E4%B8%AD%E6%96%87%E5%A1%AB%E7%A9%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#中文填空任务

*    数据集还是ChnSentiCorp,截取长度为30
*    model使用还是bert-base-chinese

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 3.5 MB 11.9 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 45.6 MB/s 
     |████████████████████████████████| 6.8 MB 36.6 MB/s 
     |████████████████████████████████| 895 kB 39.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 11.6 MB/s 
     |████████████████████████████████| 243 kB 51.3 MB/s 
     |████████████████████████████████| 1.1 MB 44.6 MB/s 
     |████████████████████████████████| 133 kB 54.0 MB/s 
     |████████████████████████████████| 271 kB 39.7 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 41.3 MB/s 


In [2]:
import torch
from datasets import load_dataset
from torch.utils.data import Dataset,DataLoader

class SentiDataset(Dataset):
  def __init__(self,split):
    dataset=load_dataset(path='seamew/ChnSentiCorp',split=split)

    def f(data):
      return len(data['text'])>30
    self.dataset=dataset.filter(f)

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self,i):
    return self.dataset[i]['text']

dataset=SentiDataset('train')

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/3.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset chn_senti_corp downloaded and prepared to /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85. Subsequent calls will reuse this data.


  0%|          | 0/10 [00:00<?, ?ba/s]

In [3]:
#分词批处理数据
from transformers import BertTokenizer,BertModel

tokenizer=BertTokenizer.from_pretrained('bert-base-chinese')


def collate_fn(data):
  data=tokenizer.batch_encode_plus(batch_text_or_text_pairs=data,
                  truncation=True,
                  padding='max_length',
                  max_length=30,
                  return_tensors='pt',
                  return_length=True)
  
  input_ids=data['input_ids']
  labels=input_ids[:,15].reshape(-1).clone()
  input_ids[:,15]=tokenizer.get_vocab()[tokenizer.mask_token]

  return input_ids,data['attention_mask'],data['token_type_ids'],labels

dataloader=DataLoader(
    dataset=dataset,
    batch_size=16,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True
)

for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(dataloader):
  break

print(tokenizer.decode(input_ids[0]))
print(tokenizer.decode(labels[0]))

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

[CLS] 31 号 服 务 员 成 心 坑 人 ！ ！ 我 们 12 [MASK] 回 宾 馆 准 备 吃 饭 后 收 拾 结 帐 ， [SEP]
点


In [17]:
#加载模型并训练
from transformers import AdamW
from torch import nn

pre=BertModel.from_pretrained('bert-base-chinese')

for param in pre.parameters():
  param.requires_grad_(False)

class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.decoder=torch.nn.Linear(768,tokenizer.vocab_size,bias=False)
    self.bias=torch.nn.Parameter(torch.zeros(tokenizer.vocab_size))
    self.decoder.bias=self.bias

  def forward(self,input_ids,attention_mask,token_type_ids,device):
    with torch.no_grad():
      pre.to(device)
      out=pre(input_ids,attention_mask,token_type_ids)

    out=self.decoder(out.last_hidden_state[:,15])
    return out

model=Model()
optimizer=AdamW(model.parameters(),lr=5e-4)
criterion=nn.CrossEntropyLoss()
device='cuda' if torch.cuda.is_available() else 'cpu'
print('device is ',device)

#验证集进行测试
def test(epoch):
  total=0
  correct=0


  test_dataset=SentiDataset('validation')
  test_loader=DataLoader(test_dataset,batch_size=32,collate_fn=collate_fn,drop_last=True)

  model.eval().to(device)

  with torch.no_grad():
    for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(test_loader):
      input_ids=input_ids.to(device)
      attention_mask=attention_mask.to(device)
      token_type_ids=token_type_ids.to(device)
      labels=labels.to(device)

      out=model(input_ids,attention_mask,token_type_ids,device)
      out=out.argmax(dim=1)
      total+=len(labels)
      correct+=(out==labels).sum().item()

  print('Epoch {} Test Accuarcy {} '.format(epoch,correct/total))


#训练

for epoch in range(5):
  model.train().to(device)
  for i ,(input_ids,attention_mask,token_type_ids,labels) in enumerate(dataloader):
    input_ids=input_ids.to(device)
    attention_mask=attention_mask.to(device)
    token_type_ids=token_type_ids.to(device)
    labels=labels.to(device)

    out=model(input_ids,attention_mask,token_type_ids,device)

    loss=criterion(out,labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if i%50==0:
      out=out.argmax(dim=1)
      accuarcy=(out==labels).sum().item()/len(labels)
      print('Epoch {} Step {} Loss {} Accuarcy {}'.format(epoch+1,i,loss.item(),accuarcy))
  test(epoch+1)
    

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a

device is  cuda
Epoch 1 Step 0 Loss 10.029353141784668 Accuarcy 0.0
Epoch 1 Step 50 Loss 8.018394470214844 Accuarcy 0.125
Epoch 1 Step 100 Loss 6.363152980804443 Accuarcy 0.1875
Epoch 1 Step 150 Loss 6.73760461807251 Accuarcy 0.1875
Epoch 1 Step 200 Loss 4.089290618896484 Accuarcy 0.4375
Epoch 1 Step 250 Loss 6.4485273361206055 Accuarcy 0.1875
Epoch 1 Step 300 Loss 5.264184474945068 Accuarcy 0.4375
Epoch 1 Step 350 Loss 3.714693069458008 Accuarcy 0.5
Epoch 1 Step 400 Loss 3.565122127532959 Accuarcy 0.4375
Epoch 1 Step 450 Loss 4.838405609130859 Accuarcy 0.3125
Epoch 1 Step 500 Loss 2.754926919937134 Accuarcy 0.5625
Epoch 1 Step 550 Loss 4.2664594650268555 Accuarcy 0.3125


Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)
Loading cached processed dataset at /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-d0ae548a4c96fdd9.arrow


Epoch 1 Test Accuarcy 0.49464285714285716 
Epoch 2 Step 0 Loss 3.098409414291382 Accuarcy 0.5
Epoch 2 Step 50 Loss 1.330715537071228 Accuarcy 0.8125
Epoch 2 Step 100 Loss 2.467085838317871 Accuarcy 0.625
Epoch 2 Step 150 Loss 2.6397318840026855 Accuarcy 0.4375
Epoch 2 Step 200 Loss 1.93503737449646 Accuarcy 0.75
Epoch 2 Step 250 Loss 1.2871290445327759 Accuarcy 0.75
Epoch 2 Step 300 Loss 1.4283133745193481 Accuarcy 0.875
Epoch 2 Step 350 Loss 2.242447853088379 Accuarcy 0.625
Epoch 2 Step 400 Loss 1.4468510150909424 Accuarcy 0.625
Epoch 2 Step 450 Loss 2.110555410385132 Accuarcy 0.625
Epoch 2 Step 500 Loss 1.1017084121704102 Accuarcy 0.875
Epoch 2 Step 550 Loss 1.2402492761611938 Accuarcy 0.6875


Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)
Loading cached processed dataset at /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-d0ae548a4c96fdd9.arrow


...

Epoch 3 Test Accuarcy 0.6589285714285714 
Epoch 4 Step 0 Loss 0.7961668968200684 Accuarcy 0.75
Epoch 4 Step 50 Loss 0.3536894619464874 Accuarcy 0.9375
Epoch 4 Step 100 Loss 0.601102888584137 Accuarcy 0.875
Epoch 4 Step 150 Loss 0.5220973491668701 Accuarcy 0.875
Epoch 4 Step 200 Loss 0.2796095013618469 Accuarcy 1.0
Epoch 4 Step 250 Loss 0.4030483365058899 Accuarcy 1.0
Epoch 4 Step 300 Loss 0.5973972678184509 Accuarcy 0.8125
Epoch 4 Step 350 Loss 0.6321725249290466 Accuarcy 0.8125
Epoch 4 Step 400 Loss 0.3442709147930145 Accuarcy 0.875
Epoch 4 Step 450 Loss 0.3424728512763977 Accuarcy 0.875
Epoch 4 Step 500 Loss 0.5596597790718079 Accuarcy 0.8125
Epoch 4 Step 550 Loss 0.8564627766609192 Accuarcy 0.75


Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)
Loading cached processed dataset at /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-d0ae548a4c96fdd9.arrow


Epoch 4 Test Accuarcy 0.6839285714285714 
Epoch 5 Step 0 Loss 0.306395560503006 Accuarcy 1.0
Epoch 5 Step 50 Loss 0.5316659212112427 Accuarcy 0.875
Epoch 5 Step 100 Loss 0.6659610867500305 Accuarcy 0.75
Epoch 5 Step 150 Loss 0.6847471594810486 Accuarcy 0.8125
Epoch 5 Step 200 Loss 0.6260770559310913 Accuarcy 0.875
Epoch 5 Step 250 Loss 0.572501003742218 Accuarcy 0.875
Epoch 5 Step 300 Loss 0.2912217080593109 Accuarcy 0.9375
Epoch 5 Step 350 Loss 0.28406980633735657 Accuarcy 0.9375
Epoch 5 Step 400 Loss 0.36160916090011597 Accuarcy 0.9375
Epoch 5 Step 450 Loss 0.49567529559135437 Accuarcy 0.9375
Epoch 5 Step 500 Loss 0.5012670159339905 Accuarcy 0.9375
Epoch 5 Step 550 Loss 0.3671070635318756 Accuarcy 0.875


Using custom data configuration default
Reusing dataset chn_senti_corp (/root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85)
Loading cached processed dataset at /root/.cache/huggingface/datasets/seamew___chn_senti_corp/default/0.0.0/1f242195a37831906957a11a2985a4329167e60657c07dc95ebe266c03fdfb85/cache-d0ae548a4c96fdd9.arrow


Epoch 5 Test Accuarcy 0.6928571428571428 


#预测并打印结果

In [27]:
input_ids,attention_mask,token_type_ids,labels=collate_fn(['我们第一次来这个宾馆，有房间不干净，服务员的态度也不是很友好。'])
input_ids=input_ids.to(device)
attention_mask=attention_mask.to(device)
token_type_ids=token_type_ids.to(device)
labels=labels.to(device)
print(tokenizer.decode(input_ids[0]))
print('正确结果:',tokenizer.decode(labels))

predict=model(input_ids,attention_mask,token_type_ids,device)
predict=predict.argmax(dim=1)
print('预测的结果:',tokenizer.decode(predict))

[CLS] 我 们 第 一 次 来 这 个 宾 馆 ， 有 房 间 [MASK] 干 净 ， 服 务 员 的 态 度 也 不 是 很 [SEP]
正确结果: 不
预测的结果: 很
